<a href="https://colab.research.google.com/github/michaelknorr/CHE4061/blob/main/CHE4061_Group_Tutorial_2_(Phase_Equilibrium).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 **Outline**
 1. **Part A**: Calculate Boiling Point for a Given Pressure
 2. **Part B**: Calculate y vs. x for a single component
 3. **Part C**: Calculate Bubble & Dew Point curves for a two component mixture (T-x)
 4. **Part D**: Calculate Relative Volatility and K
 5. **Part E**: Calculate y using alpha

In [1]:
pip install Chetoolbox

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.2 MB/s eta 0:00:00


In [16]:
import chetoolbox
from chetoolbox import separations
import numpy as np
import numpy.typing as npt
from typing import Callable
!pip install chetoolbox
import chetoolbox.separations as che
import chetoolbox.common

**Part A**: Calculate Boiling Point for a Given *Pressure*

In [17]:
def antoine_T(coeffs: npt.NDArray, P: npt.NDArray) -> npt.NDArray:
  '''
  Calculates the temperature in K for each set of vapor pressure in bar.
  '''
  coeffs = np.atleast_2d(coeffs); P = np.c_[np.atleast_1d(P)]
  return (-coeffs[:, 1] / (np.log10(P) - coeffs[:, 0])) - coeffs[:, 2]

In [18]:
antoine_T(coeffs=[7.96681, 1668.21, 228], P=[760])

array([[100.00062491]])

In [19]:
1554.3/(8.04494-np.log10(760))-222.65

78.33023891837163

**Part B**: Calculate y vs. x for a single component

In [46]:
import numpy as np
import plotly.graph_objects as go

def plot_vle_curve(coeffs_chemical_1, coeffs_chemical_2, Ptot):
    """
    Plots the Vapor-Liquid Equilibrium (VLE) curve for two chemicals using the Antoine equation.
    Also prints a table of 10 sample (x, y) values.

    Parameters:
    coeffs_chemical_1 (list): Antoine constants [A, B, C] for the first chemical.
    coeffs_chemical_2 (list): Antoine constants [A, B, C] for the second chemical.
    Ptot (float): Total system pressure in bar.

    Returns:
    None: Displays the VLE curve plot.
    """

    # Extracting Antoine constants for both chemicals
    A1, B1, C1 = coeffs_chemical_1
    A2, B2, C2 = coeffs_chemical_2

    # Function to calculate boiling point at system pressure (1 bar)
    def antoine_T(coeffs, P):
        A, B, C = coeffs
        return B / (A - np.log10(P)) - C

    # Calculate Boiling Points at System Pressure (1 bar)
    T_bp_chemical_1 = antoine_T([A1, B1, C1], Ptot)
    T_bp_chemical_2 = antoine_T([A2, B2, C2], Ptot)

    # Function to calculate vapor pressure using the Antoine equation
    def antoine_P(A, B, C, T):
        return 10 ** (A - B / (C + T))

    # Temperature range for the plot (from the boiling point of the first chemical to the second one)
    Tplot = np.linspace(T_bp_chemical_1, T_bp_chemical_2, 1000)

    # Calculate vapor pressures for both substances
    Pvap1 = antoine_P(A1, B1, C1, Tplot)
    Pvap2 = antoine_P(A2, B2, C2, Tplot)

    # Calculate mole fractions using Raoult's Law
    xplot = (1 - Pvap2 / Ptot) / (Pvap1 / Ptot - Pvap2 / Ptot)
    yplot = Pvap1 / Ptot * xplot

    # Make sure mole fractions are within valid range [0, 1]
    xplot = np.clip(xplot, 0, 1)
    yplot = np.clip(yplot, 0, 1)

    # Select 10 evenly spaced sample points for the table
    indices = np.linspace(0, len(xplot) - 1, 10, dtype=int)
    x_sample = xplot[indices]
    y_sample = yplot[indices]

    # Print table
    print(f"{'x (Liquid Mole Fraction)':<25}{'y (Vapor Mole Fraction)':<25}")
    print("-" * 50)
    for x, y in zip(x_sample, y_sample):
        print(f"{x:<25.4f}{y:<25.4f}")

    # Create the plot
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=xplot, y=yplot, mode='lines', name='VLE Curve', line=dict(color='blue')))

    # Add 45-degree reference line (y = x)
    fig.add_trace(go.Scatter(x=xplot, y=xplot, mode='lines', name='y = x (Ideal)', line=dict(color='red', dash='dash')))

    fig.update_layout(
        title="Vapor-Liquid Equilibrium Curve",
        xaxis_title="Liquid Mole Fraction (x)",
        yaxis_title="Vapor Mole Fraction (y)",
        width=800,
        height=600,
        template='plotly_dark'
    )

    fig.show()


# Example usage
coeffs_isopropanol = [8.04494, 1554.3, 222.65]
coeffs_water = [7.96681, 1668.21, 228]
Ptot = 760  # mmHg

plot_vle_curve(coeffs_isopropanol, coeffs_water, Ptot)


x (Liquid Mole Fraction) y (Vapor Mole Fraction)  
--------------------------------------------------
1.0000                   1.0000                   
0.8397                   0.9228                   
0.6952                   0.8384                   
0.5647                   0.7462                   
0.4467                   0.6459                   
0.3397                   0.5367                   
0.2425                   0.4181                   
0.1541                   0.2896                   
0.0736                   0.1504                   
0.0000                   0.0000                   


**Part C**: Calculate Bubble & Dew Point curves for a two component mixture (T-x)



In [45]:
import numpy as np
import plotly.graph_objects as go

# Antoine constants for Hexane (Lower BP)
A_Lbp = 4.0266
B_Lbp = 1171.53
C_Lbp = -48.784

# Antoine constants for Heptane (Higher BP)
A_Hbp = 4.0282
B_Hbp = 1268.636
C_Hbp = -56.199

# Function to calculate boiling point at system pressure in Celsius
def antoine_T(coeffs, P):
    A, B, C = coeffs
    T_kelvin = B / (A - np.log10(P)) - C  # Temperature in Kelvin
    T_celsius = T_kelvin - 273.15  # Convert to Celsius
    return T_celsius

# Calculate Boiling Points at System Pressure (1 bar) in Celsius
T_bp_hexane = antoine_T([A_Lbp, B_Lbp, C_Lbp], 1)
T_bp_heptane = antoine_T([A_Hbp, B_Hbp, C_Hbp], 1)

# Function to calculate vapor pressure using the Antoine equation
def antoine_P(A, B, C, T):
    return 10 ** (A - B / (C + T))

# Define system pressure (in bar)
Ptot = 1  # System pressure in bar

# Temperature range for the plot (from the boiling point of Hexane to Heptane)
Tplot = np.linspace(T_bp_hexane, T_bp_heptane, 1000)  # Temperature range

# Calculate vapor pressures for both substances
PvapA = antoine_P(A_Lbp, B_Lbp, C_Lbp, Tplot + 273.15)  # Hexane vapor pressure (convert T to Kelvin)
PvapB = antoine_P(A_Hbp, B_Hbp, C_Hbp, Tplot + 273.15)  # Heptane vapor pressure (convert T to Kelvin)

# Calculate mole fractions using Raoult's Law
xplot = (1 - PvapB / Ptot) / (PvapA / Ptot - PvapB / Ptot)
yplot = (PvapA / Ptot) * xplot

# Make sure mole fractions are within valid range [0, 1]
xplot = np.clip(xplot, 0, 1)
yplot = np.clip(yplot, 0, 1)

# Select 10 evenly spaced sample points for the table
indices = np.linspace(0, len(Tplot) - 1, 10, dtype=int)
T_sample = Tplot[indices]
x_sample = xplot[indices]
y_sample = yplot[indices]

# Print table
print(f"{'T (°C)':<10}{'x (Liquid Mole Fraction)':<25}{'y (Vapor Mole Fraction)':<25}")
print("-" * 60)
for T, x, y in zip(T_sample, x_sample, y_sample):
    print(f"{T:<10.2f}{x:<25.4f}{y:<25.4f}")

# Plot the equilibrium curve (T-x-y diagram)
fig = go.Figure()

fig.add_trace(go.Scatter(x=xplot, y=Tplot, mode='lines', name='Liquid Temperature (T-x)', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=yplot, y=Tplot, mode='lines', name='Vapor Temperature (T-y)', line=dict(color='red')))

# Update layout with the Celsius scale on the y-axis
fig.update_layout(
    title="T-x-y Plot for Mixture",
    xaxis_title="Mole Fraction (x or y)",
    yaxis_title="Temperature (°C)",  # In Celsius
    width=800,
    height=600,
    template='plotly_dark'
)

fig.show()


T (°C)    x (Liquid Mole Fraction) y (Vapor Mole Fraction)  
------------------------------------------------------------
66.58     1.0000                   1.0000                   
70.07     0.8365                   0.9337                   
73.56     0.6906                   0.8582                   
77.05     0.5600                   0.7727                   
80.54     0.4424                   0.6764                   
84.03     0.3364                   0.5684                   
87.52     0.2403                   0.4477                   
91.01     0.1529                   0.3135                   
94.50     0.0731                   0.1646                   
97.99     0.0000                   0.0000                   


In [22]:
che.dew_temp_antoine([0.4, 0.6], [[8.04494, 1554.3, 222.65], [7.96681, 1668.21, 228.0]], 760 ) #Antoine Coefficients have to be based on K scale, P in Bar

array([93.12859182])

In [23]:
che.bubble_temp_antoine([0.4, 0.6], [[8.04494, 1554.3, 222.65], [7.96681, 1668.21, 228.0]], 760 ) #Antoine Coefficients have to be based on K scale, P in Bar

array([88.98372565])

**Part D**: Calculate Relative Volatility and K

In [44]:
import numpy as np
import plotly.graph_objects as go

def plot_relative_volatility_and_K(Ptot, A1, B1, C1, A2, B2, C2):
    """
    Function to plot relative volatility (α) and K-values as a function of temperature.
    Also prints a table of 10 sample (T, K1, K2, α) values.

    Parameters:
    Ptot (float): Total system pressure in bar
    A1, B1, C1 (floats): Antoine coefficients for substance 1 (more volatile)
    A2, B2, C2 (floats): Antoine coefficients for substance 2 (less volatile)
    """

    # Antoine equation to calculate vapor pressure (P in bar, T in Celsius)
    def antoine_P(A, B, C, T):
        return np.exp(A - B / (C + T))

    # Define temperature range in Celsius
    T_C = np.linspace(99, 110, 300)  # Temperature in Celsius

    # Compute vapor pressures
    Pvap1 = antoine_P(A1, B1, C1, T_C)  # More volatile component
    Pvap2 = antoine_P(A2, B2, C2, T_C)  # Less volatile component

    # Compute equilibrium ratios (K-values)
    K1 = Pvap1 / Ptot
    K2 = Pvap2 / Ptot

    # Compute relative volatility α (α = K1/K2)
    alpha = K1 / K2

    # Select 10 evenly spaced sample points for the table
    indices = np.linspace(0, len(T_C) - 1, 10, dtype=int)
    T_sample = T_C[indices]
    K1_sample = K1[indices]
    K2_sample = K2[indices]
    alpha_sample = alpha[indices]

    # Print table
    print(f"{'T (°C)':<10}{'K1':<15}{'K2':<15}{'α (Relative Volatility)':<20}")
    print("-" * 60)
    for t, k1, k2, a in zip(T_sample, K1_sample, K2_sample, alpha_sample):
        print(f"{t:<10.2f}{k1:<15.4f}{k2:<15.4f}{a:<20.4f}")

    # Create plots
    fig = go.Figure()

    # Plot K-values
    fig.add_trace(go.Scatter(x=T_C, y=K1, mode='lines', name='K1 (Substance 1)', line=dict(color='purple')))
    fig.add_trace(go.Scatter(x=T_C, y=K2, mode='lines', name='K2 (Substance 2)', line=dict(color='red')))

    # Plot relative volatility
    fig.add_trace(go.Scatter(x=T_C, y=alpha, mode='lines', name='Relative Volatility (α)', line=dict(color='green', dash='dash')))

    # Layout settings
    fig.update_layout(
        title="Relative Volatility & K-values vs Temperature",
        xaxis_title="Temperature (°C)",
        yaxis_title="K-values & α",
        width=800,
        height=600,
        template='plotly_dark'
    )
    fig.show()

    print(f'\nThe average relative volatility (alpha) value is: {np.mean(alpha):.4f}')

# Example usage
Ptot = 760  # System pressure in torr
A1, B1, C1 = 15.7831, 2855.27, 213.64  # Antoine Coefficients for n-Heptane (More Volatile)
A2, B2, C2 = 17.2741, 3896.3, 255.67  # Antoine Coefficients for Toluene (Less Volatile)

plot_relative_volatility_and_K(Ptot, A1, B1, C1, A2, B2, C2)


T (°C)    K1             K2             α (Relative Volatility)
------------------------------------------------------------
99.00     1.0172         0.7083         1.4361              
100.21    1.0537         0.7353         1.4330              
101.43    1.0913         0.7632         1.4299              
102.64    1.1300         0.7920         1.4268              
103.86    1.1696         0.8216         1.4237              
105.11    1.2117         0.8530         1.4204              
106.32    1.2535         0.8845         1.4173              
107.54    1.2966         0.9169         1.4141              
108.75    1.3407         0.9503         1.4109              
110.00    1.3874         0.9856         1.4076              



The average relative volatility (alpha) value is: 1.4219


**Part E**: Calculate y using alpha

In [43]:
import numpy as np
import plotly.graph_objects as go

def plot_y_vs_x(Ptot, A1, B1, C1, A2, B2, C2):
    """
    Function to plot vapor mole fraction (y) vs liquid mole fraction (x),
    using the average relative volatility (alpha). Also prints a table of 10 sample (x, y) values.

    Parameters:
    Ptot (float): Total system pressure (same units as Antoine equation)
    A1, B1, C1 (floats): Antoine coefficients for more volatile component
    A2, B2, C2 (floats): Antoine coefficients for less volatile component
    """

    # Antoine equation to calculate vapor pressure (P in bar, T in Celsius)
    def antoine_P(A, B, C, T):
        return np.exp(A - B / (C + T))

    # Define temperature range in Celsius
    T_C = np.linspace(99, 110, 300)  # Temperature range in Celsius

    # Compute vapor pressures
    Pvap1 = antoine_P(A1, B1, C1, T_C)  # More volatile component
    Pvap2 = antoine_P(A2, B2, C2, T_C)  # Less volatile component

    # Compute equilibrium ratios (K-values)
    K1 = Pvap1 / Ptot
    K2 = Pvap2 / Ptot

    # Compute relative volatility α (α = K1/K2)
    alpha = K1 / K2
    avg_alpha = np.mean(alpha)  # Use average alpha for calculations

    # Define x values (liquid mole fraction, from 0 to 1)
    x = np.linspace(0, 1, 300)

    # Compute y values (vapor mole fraction) using the equilibrium relation
    y = (avg_alpha * x) / (1 + x * (avg_alpha - 1))

    # Select 10 evenly spaced values for the table
    x_sample = np.linspace(0, 1, 11)
    y_sample = (avg_alpha * x_sample) / (1 + x_sample * (avg_alpha - 1))

    # Print table
    print(f"{'x (Liquid Mole Fraction)':<25}{'y (Vapor Mole Fraction)':<25}")
    print("-" * 50)
    for xi, yi in zip(x_sample, y_sample):
        print(f"{xi:<25.4f}{yi:<25.4f}")

    # Create plot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='Equilibrium Curve', line=dict(color='blue')))

    # Add 45-degree line (y = x) for reference
    fig.add_trace(go.Scatter(x=x, y=x, mode='lines', name='y = x', line=dict(color='red', dash='dash')))

    # Layout settings
    fig.update_layout(
        title="Equilibrium Curve (y vs x) Using Average Alpha",
        xaxis_title="Liquid Mole Fraction (x)",
        yaxis_title="Vapor Mole Fraction (y)",
        width=800,
        height=600,
        template='plotly_dark'
    )
    fig.show()

# Example usage
Ptot = 760  # System pressure in torr
A1, B1, C1 = 15.7831, 2855.27, 213.64  # Antoine Coefficients for n-Heptane (More Volatile)
A2, B2, C2 = 17.2741, 3896.3, 255.67  # Antoine Coefficients for Toluene (Less Volatile)

plot_y_vs_x(Ptot, A1, B1, C1, A2, B2, C2)


x (Liquid Mole Fraction) y (Vapor Mole Fraction)  
--------------------------------------------------
0.0000                   0.0000                   
0.1000                   0.1364                   
0.2000                   0.2623                   
0.3000                   0.3787                   
0.4000                   0.4866                   
0.5000                   0.5871                   
0.6000                   0.6808                   
0.7000                   0.7684                   
0.8000                   0.8505                   
0.9000                   0.9275                   
1.0000                   1.0000                   
